<a href="https://colab.research.google.com/github/Black2oo4g/GoogleColab/blob/main/DATAACOMEMOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
import pandas as pd
import json

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Asegurarse que columnas estén bien
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Definir meses y su número
meses_mapping = {
    "ENERO": "01",
    "FEBRERO": "02",
    "MARZO": "03",
    "ABRIL": "04",
    "MAYO": "05",
    "JUNIO": "06",
    "JULIO": "07",
    "AGOSTO": "08",
    "SEPTIEMBRE": "09",
    "OCTUBRE": "10",
    "NOVIEMBRE": "11",
    "DICIEMBRE": "12"
}

# 8. Crear JSON
cols = df_energy.columns.tolist()
json_list = []

for _, row in df_energy.iterrows():
    cod = row["ID"]
    if pd.isna(cod):
        continue
    try:
        cod_int = int(cod)
    except (ValueError, TypeError):
        continue
    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre, mes_num in meses_mapping.items():
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue
                emi_date = pd.to_datetime(emi_raw, dayfirst=True, errors="coerce")
                if pd.isna(emi_date):
                    continue
                venc_date = emi_date + pd.Timedelta(days=30)

                # El año lo sacamos de emi_date
                year = emi_date.strftime("%Y")

                # El mes es basado en el nombre de la columna
                periodo = f"{year}-{mes_num}"

                json_list.append({
                    "cod": cod_int,
                    "periodo": periodo,
                    "monto": float(monto) if pd.notna(monto) else 0.0,
                    "emi": emi_date.strftime("%Y-%m-%d"),
                    "venc": venc_date.strftime("%Y-%m-%d"),
                    "operacion": str(operacion)
                })

# 9. Guardar JSON
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/salida_pagos.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 10. Guardar socios no reconocidos
output_unrecognized_path = "/content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos.xlsx"
df_unrecognized.to_excel(output_unrecognized_path, index=False)

# 11. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print("Total registros NO RECONOCIDOS (sin ID):", len(df_unrecognized))
print(f"Archivo de socios no reconocidos guardado en: {output_unrecognized_path}")

# 12. Mostrar ejemplo de JSON generado
print(json.dumps(json_list[:5], indent=4, ensure_ascii=False))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total registros RECONOCIDOS (con ID): 254
Total registros NO RECONOCIDOS (sin ID): 169
Archivo de socios no reconocidos guardado en: /content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos.xlsx
[
    {
        "cod": 529,
        "periodo": "2025-01",
        "monto": 116.6,
        "emi": "2025-04-03",
        "venc": "2025-05-03",
        "operacion": "17399"
    },
    {
        "cod": 529,
        "periodo": "2025-02",
        "monto": 120.8,
        "emi": "2025-03-26",
        "venc": "2025-04-25",
        "operacion": "17709"
    },
    {
        "cod": 529,
        "periodo": "2025-03",
        "monto": 135.6,
        "emi": "2025-04-23",
        "venc": "2025-05-23",
        "operacion": "18106.0"
    },
    {
        "cod": 18,
        "periodo": "2025-01",
        "monto": 34.5,
        "emi": "2025-03-03",
        "venc": "2025-04-02",
       

In [ ]:
from google.colab import drive
import pandas as pd
import json

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrames de reconocidos y no reconocidos
df_recognized = df_energy.dropna(subset=["ID"])
df_unrecognized = df_energy[df_energy["ID"].isna()]

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]
meses_mapping = {mes: str(i+1).zfill(2) for i, mes in enumerate(meses)}

# 9. Crear JSON agrupado por cod
cols = df_energy.columns.tolist()
json_dict = {}

for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    first_emi_found = False
    base_year = None
    mes_idx = 0

    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre in meses:
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue

                # Convertir la fecha de pago directamente sin cálculos
                emi_date = pd.to_datetime(emi_raw, dayfirst=True, errors="coerce")
                if pd.isna(emi_date) or pd.isna(monto):
                    continue

                # Definir periodo
                periodo = f"{emi_date.year}-{str(mes_idx + 1).zfill(2)}"

                pago = {
                    "periodo": periodo,
                    "monto": float(monto),
                    "fecha_pago": emi_date.strftime("%Y-%m-%d"),  # Usar la fecha de pago directamente
                    "operacion": str(operacion)
                }

                if cod not in json_dict:
                    json_dict[cod] = []

                json_dict[cod].append(pago)
                mes_idx += 1

# 10. Guardar JSON agrupado
import os
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/energia-2025.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_dict, f, indent=4, ensure_ascii=False)

# 11. Guardar socios no reconocidos
output_unrecognized_path = "/content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos-2025.xlsx"
df_unrecognized.to_excel(output_unrecognized_path, index=False)

# 12. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print("Total registros NO RECONOCIDOS (sin ID):", len(df_unrecognized))
print(f"Archivo actualizado: {energy_path}")
print(f"Archivo de socios no reconocidos: {output_unrecognized_path}")

# 13. Mostrar un ejemplo del JSON generado (primer comerciante)
first_cod = next(iter(json_dict))
print(f"Ejemplo de pagos para el COD {first_cod}:")
print(json.dumps(json_dict[first_cod], indent=4, ensure_ascii=False))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total registros RECONOCIDOS (con ID): 395
Total registros NO RECONOCIDOS (sin ID): 28
Archivo actualizado: /content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx
Archivo de socios no reconocidos: /content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos-2025.xlsx
Ejemplo de pagos para el COD 529:
[
    {
        "periodo": "2025-01",
        "monto": 116.6,
        "fecha_pago": "2025-04-03",
        "operacion": "17399"
    },
    {
        "periodo": "2025-02",
        "monto": 120.8,
        "fecha_pago": "2025-03-26",
        "operacion": "17709"
    },
    {
        "periodo": "2025-03",
        "monto": 135.6,
        "fecha_pago": "2025-04-23",
        "operacion": "18106.0"
    }
]


In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]

# 9. Crear JSON de pagos para todos los comerciantes (sin agrupar)
json_list = []

cols = df_energy.columns.tolist()
for _, row in df_recognized.iterrows():
    cod = int(row["ID"])

    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre in meses:
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue

                # Convertir la fecha de pago directamente sin cálculos
                emi_date = pd.to_datetime(emi_raw, dayfirst=True, errors="coerce")
                if pd.isna(emi_date) or pd.isna(monto):
                    continue

                # Definir periodo
                #periodo = f"{emi_date.year}-{str(mes_idx + 1).zfill(2)}"
                mes_num = str(meses.index(mes_nombre) + 1).zfill(2)
                periodo = f"2025-{mes_num}"


                pago = {
                    "id_comerciante": cod,
                    "periodo": periodo,
                    "monto": float(monto),
                    "fecha_pago": emi_date.strftime("%Y-%m-%d"),  # Usar la fecha de pago directamente
                    "operacion": str(operacion)
                }

                json_list.append(pago)


# 10. Guardar JSON (sin agrupar por ID)
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/energia-2025.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 11. Mostrar solo los primeros 5 registros
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 12. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
#print("Total registros NO RECONOCIDOS (sin ID):", len(df_unrecognized))
print(f"Archivo actualizado: {energy_path}")
#print(f"Archivo de socios no reconocidos: {output_unrecognized_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 529,
        "periodo": "2025-01",
        "monto": 116.6,
        "fecha_pago": "2025-04-03",
        "operacion": "17399"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-02",
        "monto": 120.8,
        "fecha_pago": "2025-03-26",
        "operacion": "17709"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-03",
        "monto": 135.6,
        "fecha_pago": "2025-04-23",
        "operacion": "18106"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-01",
        "monto": 34.5,
        "fecha_pago": "2025-03-03",
        "operacion": "17371"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-02",
        "monto": 31.9,
        "fecha_pago": "2025-03-27",
        "operacion": "17711"
    },
    {
        "id_comerc

In [ ]:
from google.colab import drive
import pandas as pd

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"

# 3. Leer las dos hojas
df_ref = pd.read_excel(energy_path, sheet_name="PAGOS2024", header=0)
df_target = pd.read_excel(energy_path, sheet_name="PAGOS2022", header=0)

# 4. Normalizar columnas
for df in [df_ref, df_target]:
    df.columns = df.columns.str.strip()
    df["SOCIO"] = df["SOCIO"].astype(str).str.strip().str.upper()
    df["MANZANA"] = df["MANZANA"].astype(str).str.strip().str.upper()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)

# 5. Crear clave de combinación: LOTE + MANZANA + SOCIO
df_ref["CLAVE"] = df_ref.iloc[:, 0] + "|" + df_ref["MANZANA"] + "|" + df_ref["SOCIO"]
df_target["CLAVE"] = df_target.iloc[:, 0] + "|" + df_target["MANZANA"] + "|" + df_target["SOCIO"]

# 6. Crear diccionario de mapeo: CLAVE → ID
clave_id_map = df_ref.set_index("CLAVE")["ID"].dropna().to_dict()

# 7. Asignar el ID según coincidencia en clave
df_target["ID"] = df_target["CLAVE"].map(clave_id_map)

# 8. Eliminar columna auxiliar
df_ref.drop(columns=["CLAVE"], inplace=True)
df_target.drop(columns=["CLAVE"], inplace=True)

# 9. Guardar archivo actualizado
output_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021_actualizado.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    df_ref.to_excel(writer, sheet_name="PAGOS2024", index=False)
    df_target.to_excel(writer, sheet_name="PAGOS2022", index=False)

# 10. Verificación
print("IDs asignados en PAGOS2022:")
print(df_target[["SOCIO", "MANZANA", df_target.columns[0], "ID"]].head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-13-f5e0da5a9241>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
424    30.0
425    30.0
426     NAN
427     NAN
428     NAN
Name: Unnamed: 0, Length: 429, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)
<ipython-input-13-f5e0da5a9241>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
424    30.0
425    30.0
426     NAN
427     NAN
428    13.0
Name: , Length: 429, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Pri

IDs asignados en PAGOS2022:
                               SOCIO MANZANA          ID
0          SILVA SALDARRIAGA ALBERTO      12  1.0  529.0
1          CARRASCO CASTILLO ALFONSO     13A  1.0   18.0
2          VILLEGAS PACHERREZ MANUEL     13B  1.0  621.0
3  HIDALGO VDA. DE CHAPARRO HILBERTA    14-A  1.0   19.0
4  HIDALGO VDA. DE CHAPARRO HILBERTA    14-B  1.0  655.0


In [ ]:
from google.colab import drive
import pandas as pd

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"

# 3. Leer las dos hojas
df_ref = pd.read_excel(energy_path, sheet_name="PAGOS2024", header=0)
df_target = pd.read_excel(energy_path, sheet_name="PAGOS2022", header=0)

# 4. Normalizar columnas
for df in [df_ref, df_target]:
    df.columns = df.columns.str.strip()
    df["SOCIO"] = df["SOCIO"].astype(str).str.strip().str.upper()
    df["MANZANA"] = df["MANZANA"].astype(str).str.strip().str.upper()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)

# 5. Crear clave de combinación: LOTE + MANZANA + SOCIO
df_ref["CLAVE"] = df_ref.iloc[:, 0] + "|" + df_ref["MANZANA"] + "|" + df_ref["SOCIO"]
df_target["CLAVE"] = df_target.iloc[:, 0] + "|" + df_target["MANZANA"] + "|" + df_target["SOCIO"]

# 6. Crear diccionario de mapeo: CLAVE → ID
clave_id_map = df_ref.set_index("CLAVE")["ID"].dropna().to_dict()

# 7. Asignar el ID según coincidencia en clave
df_target["ID"] = df_target["CLAVE"].map(clave_id_map)

# 8. Eliminar columna auxiliar
df_ref.drop(columns=["CLAVE"], inplace=True)
df_target.drop(columns=["CLAVE"], inplace=True)

# 9. Guardar archivo actualizado
output_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021_actualizado.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    df_ref.to_excel(writer, sheet_name="PAGOS2024", index=False)
    df_target.to_excel(writer, sheet_name="PAGOS2022", index=False)

# 10. Verificación
print("IDs asignados en PAGOS2022:")
print(df_target[["SOCIO", "MANZANA", df_target.columns[0], "ID"]].head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivos disponibles en el directorio de Energía Eléctrica:
['COMERCIANTESVIGLIMP.xlsx', 'no_encontrados.xlsx', 'comerciantes.xlsx', 'OUTPUT', 'socios_no_reconocidos-2025.xlsx', 'ENERGIAELECTRICA20211.xlsx', 'ENERGIAELECTRICA2021_actualizado.xlsx', 'ENERGIAELECTRICA2021.xlsx']
Hojas en el archivo de energía eléctrica:
['MED.2025', 'PAGOS25', 'MED.2024', 'PAGOS2024', 'MED.2023', 'PAGOS2023', 'Sheet4', 'CONVENIOS', 'MED.2022', 'PAGOS2022', 'MED21', '2021', 'PAGOS-2020', 'PAGOS-2019', 'Sheet12', 'Sheet3', 'RECIBOS', 'Sin MED', 'Sheet11']
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 529,
        "periodo": "2022-01",
        "monto": 112.5,
        "fecha_pago": "02/10/2022",
        "operacion": "3436"
    },
    {
        "id_comerciante": 529,
        "periodo": "2022-02",
        "monto": 124.5,
        "fecha_pago": "15/03/2022",
     

In [ ]:
from google.colab import drive
import pandas as pd

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"

# 3. Leer las dos hojas
df_ref = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_target = pd.read_excel(energy_path, sheet_name="MED2025", header=0)

# 4. Normalizar columnas
for df in [df_ref, df_target]:
    df.columns = df.columns.str.strip()
    df["SOCIO"] = df["SOCIO"].astype(str).str.strip().str.upper()
    df["MANZANA"] = df["MANZANA"].astype(str).str.strip().str.upper()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)

# 5. Crear clave de combinación: LOTE + MANZANA + SOCIO
df_ref["CLAVE"] = df_ref.iloc[:, 0] + "|" + df_ref["MANZANA"] + "|" + df_ref["SOCIO"]
df_target["CLAVE"] = df_target.iloc[:, 0] + "|" + df_target["MANZANA"] + "|" + df_target["SOCIO"]

# 6. Crear diccionario de mapeo: CLAVE → ID
clave_id_map = df_ref.set_index("CLAVE")["ID"].dropna().to_dict()

# 7. Asignar el ID según coincidencia en clave
df_target["ID"] = df_target["CLAVE"].map(clave_id_map)

# 8. Eliminar columna auxiliar
df_ref.drop(columns=["CLAVE"], inplace=True)
df_target.drop(columns=["CLAVE"], inplace=True)

# 9. Guardar archivo actualizado
output_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAFAC2025.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    df_ref.to_excel(writer, sheet_name="PAGOS25", index=False)
    df_target.to_excel(writer, sheet_name="MED2025", index=False)

# 10. Verificación
print("IDs asignados en PAGOS2025:")
print(df_target[["SOCIO", "MANZANA", df_target.columns[0], "ID"]].head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyError: 'MANZANA'

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS2025", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]

# 9. Crear JSON de pagos para todos los comerciantes (sin agrupar)
json_list = []

cols = df_energy.columns.tolist()
for _, row in df_recognized.iterrows():
    cod = int(row["ID"])

    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre in meses:
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue

                # Convertir la fecha de pago correctamente con mes/día/año (sin error)
                emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=False)

                if pd.isna(emi_date) or pd.isna(monto):
                    continue

                # Definir periodo
                mes_num = str(meses.index(mes_nombre) + 1).zfill(2)
                periodo = f"2025-{mes_num}"

                pago = {
                    "id_comerciante": cod,
                    "periodo": periodo,
                    "monto": float(monto),
                    "fecha_pago": emi_date.strftime("%m/%d/%Y"),  # Cambiado a mes/día/año
                    "operacion": str(operacion)
                }

                json_list.append(pago)

# 10. Guardar JSON (sin agrupar por ID)
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/energia-2025.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 11. Mostrar solo los primeros 5 registros
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 12. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print(f"Archivo actualizado: {energy_path}")


### **VIGILANCIAAAA**

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/COMERCIANTESVIGLIMP.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="2025", header=0)
df_energy.columns = df_energy.columns.str.strip().str.upper()
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]

# 9. Crear JSON de pagos
json_list = []
cols = df_energy.columns.tolist()

for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    for i in range(len(cols)):
        col_upper = str(cols[i]).strip().upper()
        for mes_nombre in meses:
            if mes_nombre in col_upper:
                try:
                    monto_raw = row[cols[i]]
                    emi_raw = row[cols[i + 1]] if i + 1 < len(cols) else None
                    operacion_raw = row[cols[i + 2]] if i + 2 < len(cols) else ""
                except Exception:
                    continue

                # Validar que monto_raw no sea una Serie
                if isinstance(monto_raw, pd.Series) or isinstance(emi_raw, pd.Series):
                    continue

                try:
                    monto = float(monto_raw)
                except:
                    continue

                emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)
                if pd.isna(monto) or pd.isna(emi_date):
                    continue

                mes_num = str(meses.index(mes_nombre) + 1).zfill(2)
                periodo = f"2025-{mes_num}"

                pago = {
                    "id_comerciante": cod,
                    "periodo": periodo,
                    "monto": monto,
                    "fecha_pago": emi_date.strftime("%d/%m/%Y"),
                    "operacion": str(operacion_raw)
                }

                json_list.append(pago)

# 10. Guardar JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "vigilancia-2025.json")
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 11. Mostrar ejemplo
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 12. Logs
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print(f"Archivo actualizado: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 529,
        "periodo": "2025-02",
        "monto": 60.0,
        "fecha_pago": "04/03/2025",
        "operacion": "19457"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-03",
        "monto": 60.0,
        "fecha_pago": "26/03/2025",
        "operacion": "19741"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-04",
        "monto": 60.0,
        "fecha_pago": "23/04/2025",
        "operacion": "20106"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-02",
        "monto": 60.0,
        "fecha_pago": "27/03/2025",
        "operacion": "19743"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-03",
        "monto": 60.0,
        "fecha_pago": "16/04/2025",
        "operacion": "20024"
    },
    {
        "id_comercian

# PAGOS **VIGILANCIAAAAA**

In [4]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime
from openpyxl.utils import column_index_from_string

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/COMERCIANTESVIGLIMP.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="2021", header=0)
df_energy.columns = df_energy.columns.str.strip().str.upper()

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()

# 5. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 6. Definir mapeo de columnas para cada mes
mes_a_columnas = {
    "ENERO":     {"monto":"E",  "fecha":"F",  "oper":"G"},
    "FEBRERO":   {"monto":"H",  "fecha":"I",  "oper":"J"},
    "MARZO":     {"monto":"K",  "fecha":"L",  "oper":"M"},
    "ABRIL":     {"monto":"N",  "fecha":"O",  "oper":"P"},
    "MAYO":      {"monto":"Q",  "fecha":"R",  "oper":"S"},
    "JUNIO":     {"monto":"T",  "fecha":"U",  "oper":"V"},
    "JULIO":     {"monto":"W",  "fecha":"X",  "oper":"Y"},
    "AGOSTO":    {"monto":"Z",  "fecha":"AA", "oper":"AB"},
    "SEPTIEMBRE":{"monto":"AC", "fecha":"AD", "oper":"AE"},
    "OCTUBRE":   {"monto":"AF", "fecha":"AG", "oper":"AH"},
    "NOVIEMBRE": {"monto":"AI", "fecha":"AJ", "oper":"AK"},
    "DICIEMBRE": {"monto":"AM", "fecha":"AN", "oper":"AO"},
}

# 7. Convertir las letras de columnas a índices de pandas (0 basado)
for mes, cols in mes_a_columnas.items():
    for tipo, letra in cols.items():
        idx = column_index_from_string(letra) - 1
        mes_a_columnas[mes][tipo] = idx

# 8. Crear JSON de pagos
json_list = []

for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    for mes, cols in mes_a_columnas.items():
        monto_raw = row.iloc[cols["monto"]]
        emi_raw   = row.iloc[cols["fecha"]]
        oper_raw  = row.iloc[cols["oper"]]

        try:
            monto = float(monto_raw)
        except:
            continue

        if pd.isna(monto) or monto <= 0:
            continue

        # Fecha: permitir nula y marcar como 'S/F' si falta
        emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)
        fecha_pago = emi_date.strftime("%d/%m/%Y") if not pd.isna(emi_date) else "S/F"

        # Operación: marcar como 'S/O' si falta
        operacion = str(oper_raw).strip() if pd.notna(oper_raw) and str(oper_raw).strip() else "S/O"

        mes_num = str(list(mes_a_columnas).index(mes) + 1).zfill(2)
        periodo = f"2021-{mes_num}"

        pago = {
            "id_comerciante": cod,
            "periodo": periodo,
            "monto": monto,
            "fecha_pago": fecha_pago,
            "operacion": operacion
        }
        json_list.append(pago)

# 9. Guardar JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/Vigilancia"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "vigilancia-2021.json")
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 10. Mostrar ejemplo
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 11. Logs
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print(f"Archivo actualizado: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 529,
        "periodo": "2021-01",
        "monto": 40.0,
        "fecha_pago": "S/F",
        "operacion": "999"
    },
    {
        "id_comerciante": 529,
        "periodo": "2021-03",
        "monto": 40.0,
        "fecha_pago": "S/F",
        "operacion": "1212"
    },
    {
        "id_comerciante": 529,
        "periodo": "2021-04",
        "monto": 60.0,
        "fecha_pago": "S/F",
        "operacion": "1458"
    },
    {
        "id_comerciante": 529,
        "periodo": "2021-05",
        "monto": 60.0,
        "fecha_pago": "S/F",
        "operacion": "1730"
    },
    {
        "id_comerciante": 529,
        "periodo": "2021-06",
        "monto": 60.0,
        "fecha_pago": "S/F",
        "operacion": "1901"
    },
    {
        "id_comerciante": 529,
        "periodo": "2021-07",

### ***Facturacion de ENERGIA ELECTRICA***

> Codigo para crear obtener las facturas




In [3]:
from google.colab import drive
import pandas as pd
import json
import os

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"

# 3. Leer la hoja MED2024
df_energy = pd.read_excel(energy_path, sheet_name="MED2025")

# 4. Normalizar nombres de columnas y limpiar datos
df_energy.columns = df_energy.columns.str.strip().str.upper()
df_energy["ID"] = pd.to_numeric(df_energy["ID"], errors="coerce")

# 5. Definir columnas por mes
columnas_mes = {
    "2025-01": "ENEROE",
    "2025-02": "FEBREROE",
    "2025-03": "MARZOE",
    "2025-04": "ABRILE",
    "2025-05": "MAYOE",
    "2025-06": "JUNIOE",
    "2025-07": "JULIOE",
    "2025-08": "AGOSTOE",
    "2025-09": "SEPTIEMBREE",
    "2025-10": "OCTUBREE",
    "2025-11": "NOVIEMBREE",
    "2025-12": "DICIEMBREE",
}

# 6. Crear lista para el JSON
json_list = []

for _, row in df_energy.iterrows():
    id_comerciante = row.get("ID", None)
    if pd.isna(id_comerciante):
        continue

    for periodo, nombre_mes in columnas_mes.items():
        if nombre_mes in df_energy.columns:
            monto = row.get(nombre_mes, None)

            # Verificar que el monto sea un número válido
            try:
                monto_float = float(monto)
                if monto_float > 0:  # Opcional: solo si es mayor a cero
                    json_list.append({
                        "id_comerciante": int(id_comerciante),
                        "periodo": periodo,
                        "monto": monto_float
                    })
            except (ValueError, TypeError):
                continue  # Saltar si no es un número válido

# 7. Guardar el JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/Energia"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "energia_2025_factura.json")

with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 8. Mostrar ejemplo
print("Primeros pagos exportados:")
print(json.dumps(json_list[:10], indent=4, ensure_ascii=False))
print(f"Total pagos generados: {len(json_list)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Primeros pagos exportados:
[
    {
        "id_comerciante": 529,
        "periodo": "2025-01",
        "monto": 116.6
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-02",
        "monto": 120.8
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-03",
        "monto": 135.6
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-04",
        "monto": 129.2
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-01",
        "monto": 34.6
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-02",
        "monto": 31.9
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-03",
        "monto": 35.9
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-04",
        "monto": 32.8
    },
    {
        "id_comerciante": 621,
        "periodo": "2025-01",
       

PAGOS ENERGIA **ELECTRICA**

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime
from openpyxl.utils import column_index_from_string

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"
#comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS2025", header=0)
df_energy.columns = df_energy.columns.str.strip().str.upper()
#df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
#df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
#mapping = dict(zip(
 #   df_com["SOCIO"].astype(str).str.strip().str.upper(),
  #  df_com["id_comerciante"]
#))

# 6. Rellenar ID donde esté vacío
#df_energy["ID"] = df_energy["ID"].fillna(
 #   df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
#)

# 7. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir mapeo de columnas para cada mes
mes_a_columnas = {
    "ENERO":     {"monto":"E",  "fecha":"F",  "oper":"G"},
    "FEBRERO":   {"monto":"H",  "fecha":"I",  "oper":"J"},
    "MARZO":     {"monto":"K",  "fecha":"L",  "oper":"M"},
    "ABRIL":     {"monto":"N",  "fecha":"O",  "oper":"P"},
    "MAYO":      {"monto":"Q",  "fecha":"R",  "oper":"S"},
    "JUNIO":     {"monto":"T",  "fecha":"U",  "oper":"V"},
    "JULIO":     {"monto":"W",  "fecha":"X",  "oper":"Y"},
    "AGOSTO":    {"monto":"Z",  "fecha":"AA", "oper":"AB"},
    "SEPTIEMBRE":{"monto":"AC", "fecha":"AD", "oper":"AE"},
    "OCTUBRE":   {"monto":"AF", "fecha":"AG", "oper":"AH"},
    "NOVIEMBRE": {"monto":"AI", "fecha":"AJ", "oper":"AK"},
    "DICIEMBRE": {"monto":"AM", "fecha":"AN", "oper":"AO"},
}

# 9. Convertir las letras de las columnas a índices de pandas (0 basado)
for mes, cols in mes_a_columnas.items():
    for tipo, letra in cols.items():
        # openpyxl da 1-based, pandas 0-based
        idx = column_index_from_string(letra) - 1
        mes_a_columnas[mes][tipo] = idx

# 10. Crear JSON de pagos
json_list = []

for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    for mes, cols in mes_a_columnas.items():
        monto_raw = row.iloc[cols["monto"]]
        emi_raw   = row.iloc[cols["fecha"]]
        oper_raw  = row.iloc[cols["oper"]]

        # Conversión y validación
        try:
            monto = float(monto_raw)
            emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)
        except:
            continue
        if pd.isna(monto) or pd.isna(emi_date):
            continue

        mes_num = str(list(mes_a_columnas).index(mes) + 1).zfill(2)
        periodo = f"2025-{mes_num}"

        pago = {
            "id_comerciante": cod,
            "periodo": periodo,
            "monto": monto,
            "fecha_pago": emi_date.strftime("%d/%m/%Y"),
            "operacion": str(oper_raw)
        }
        json_list.append(pago)

# 11. Guardar JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/Energia"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "energia-2025.json")
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 12. Mostrar ejemplo
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 13. Logs
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print(f"Archivo actualizado: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 529,
        "periodo": "2025-01",
        "monto": 116.6,
        "fecha_pago": "03/04/2025",
        "operacion": "17399"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-02",
        "monto": 120.8,
        "fecha_pago": "26/03/2025",
        "operacion": "17709"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-03",
        "monto": 135.6,
        "fecha_pago": "23/04/2025",
        "operacion": "18106"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-01",
        "monto": 34.5,
        "fecha_pago": "03/03/2025",
        "operacion": "17371"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-02",
        "monto": 31.9,
        "fecha_pago": "27/03/2025",
        "operacion": "17711"
    },
    {
        "id_comerc

Para combinar ID------------------------------------------------------------------------------


In [ ]:
from google.colab import drive
import pandas as pd

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"

# 3. Leer las dos hojas
df_ref = pd.read_excel(energy_path, sheet_name="PAGOS2022", header=0)
df_target = pd.read_excel(energy_path, sheet_name="MED2022", header=0)

# 4. Normalizar columnas
for df in [df_ref, df_target]:
    df.columns = df.columns.str.strip()
    df["SOCIO"] = df["SOCIO"].astype(str).str.strip().str.upper()
    df.iloc[:, 1] = df.iloc[:, 1].astype(str).str.strip().str.upper()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)

# 5. Crear clave de combinación: LOTE + MANZANA + SOCIO
df_ref["CLAVE"] = df_ref.iloc[:, 0] + "|" + df_ref.iloc[:, 1] + "|" + df_ref["SOCIO"]
df_target["CLAVE"] = df_target.iloc[:, 0] + "|" + df_target.iloc[:, 1] + "|" + df_target["SOCIO"]

# 6. Crear diccionario de mapeo: CLAVE → ID
clave_id_map = df_ref.set_index("CLAVE")["ID"].dropna().to_dict()

# 7. Asignar el ID según coincidencia en clave
df_target["ID"] = df_target["CLAVE"].map(clave_id_map)

# 8. Eliminar columna auxiliar
df_ref.drop(columns=["CLAVE"], inplace=True)
df_target.drop(columns=["CLAVE"], inplace=True)

# 9. Guardar archivo actualizado
output_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/new.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    df_ref.to_excel(writer, sheet_name="PAGOS2022", index=False)
    df_target.to_excel(writer, sheet_name="MED2022", index=False)

# 10. Verificación
print("IDs asignados en PAGOS2022:")


Mounted at /content/drive


<ipython-input-1-20412cafaca8>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
424    30.0
425    30.0
426     NAN
427     NAN
428    13.0
Name: , Length: 429, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)
<ipython-input-1-20412cafaca8>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
426    30.0
427    30.0
428    30.0
429    30.0
430     NAN
Name: Unnamed: 0, Length: 431, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Prime

IDs asignados en PAGOS2022:
